# Farina in Natura - estratégia

![](Logo_farina_in_natura_edited.jpg)

## Importando os dados e observações iniciais.

In [1]:
#.git log --pretty=format:%cI,%h,%an,%ae,%s >  C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\COMMIT_LOG\commits.csv

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import nbformat
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
from IPython.display import display
import plotly.graph_objs as go
import plotly.offline as py
from matplotlib.backends.backend_pdf import PdfPages
from fpdf import FPDF
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [56]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [57]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [58]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados.

In [59]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda            6
data_hora        6
cliente          6
mesa          3219
item            76
produto          6
quantidade       6
valor_unit       6
desconto         6
total_item       6
dtype: int64

In [60]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [61]:
# Delete the column "item"

del transactions["item"]

In [62]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [63]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
121,520.0,02/02/2022 16:57,CONSUMIDOR,M0,BOMBOLONI NUTELLA 80G,2.0,12,0.0,24
122,520.0,02/02/2022 16:57,CONSUMIDOR,M0,TARTELETE DE MORANGO 120G,1.0,18,0.0,18
123,520.0,02/02/2022 16:57,CONSUMIDOR,M0,CROISSANT MUSSATRELA DE,1.0,"25,5",0.0,"25,5"
894,856.0,11/02/2022 19:19,CONSUMIDOR,M0,CROISSANT BRIE PARMA MEL DE,2.0,"25,5",0.0,51
895,856.0,11/02/2022 19:19,CONSUMIDOR,M0,VH SANTA HORTENSIA SAUVIG,1.0,59,0.0,59


In [64]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [65]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [66]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [67]:
# change the decimal separator of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [68]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [69]:


# Convert "data_hora" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

# Creating a horario column that is a copy of data_hora and coverting it to datetime and extracting only the day;
transactions['horario'] = (transactions['data_hora'])
transactions['horario'] = transactions['horario'].dt.date

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [70]:

# https://www.dataquest.io/blog/python-datetime-tutorial/


# add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.data_hora.dt.hour
transactions['minuto'] = transactions.data_hora.dt.minute



### Creating a column name 'hora_minuto'

In [71]:
# https://gist.github.com/krisbolton/aef6179ecf906892565d326b979b3a5b
# https://www.youtube.com/watch?v=TbXaP56Omuc


transactions['hora_minuto']=pd.to_datetime(transactions['data_hora'])

transactions['hora_minuto'] = transactions['hora_minuto'].dt.time

### Renaming the column "data_hora" to "data"

In [72]:
# https://www.statology.org/pandas-rename-columns/
# renaming the column

transactions.rename(columns = {'data_hora':'data'}, inplace = True)


### Extracting only the Date from 'data' column

In [73]:

## https://datagy.io/pandas-extract-date-from-datetime/
transactions['data'] = transactions['data'].dt.normalize()

In [74]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'

In [75]:
# Apply time of day function to 'hora' column

transactions['time_of_day'] = transactions.hora.apply(time_of_day)


In [76]:
# https://pt.stackoverflow.com/questions/508484/mostrar-dias-da-semana#:~:text=Como%20weekday()%20retorna%20um,fim%20de%20semana%20ou%20n%C3%A3o.

import locale
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [77]:
# Add a day of week variable and a second classifier of weekday or weekend

from datetime import date
import calendar

transactions['day_of_week'] = [calendar.day_name[i.weekday()] for i in transactions['horario']]


In [78]:

transactions['day_type'] = ['FDS' if (i == 'domingo') | (i == 'sábado') else 'DDS' for i in 
                                                                            transactions['day_of_week']]


In [79]:
transactions.rename(columns = {'day_of_week':'dia_da_semana'}, inplace = True)

In [80]:
transactions.rename(columns = {'time_of_day':'hora_do_dia'}, inplace = True)

In [81]:
transactions.rename(columns = {'day_type':'tipo_de_dia'}, inplace = True)

In [82]:
# Add an item count variable equal to one for future groupby operations

transactions['item_count'] = 1


In [112]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,horario,hora,minuto,hora_minuto,hora_do_dia,dia_da_semana,tipo_de_dia,item_count
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,13,08:13:00,manhã,terça-feira,DDS,1
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,13,08:13:00,manhã,terça-feira,DDS,1
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,14,08:14:00,manhã,terça-feira,DDS,1
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,14,08:14:00,manhã,terça-feira,DDS,1
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,2022-02-01,9,1,09:01:00,manhã,terça-feira,DDS,1


# Verificando divergência dos dados

In [118]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [119]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


#### Creating a Column diference

In [120]:

transactions['diference']=(transactions['total_item'])-(transactions['total_item_verif'])

In [121]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True, ["total_item", "total_item_verif", 'diference']].sum()

print (resumo)

total_item          53878.3
total_item_verif    22219.1
diference           31659.2
dtype: float64


### Gerando um relatório dos dados divergentes

In [122]:

# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "data","produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [123]:
ts2.head()

,venda,data,produto,quantidade,valor_unit,total_item,total_item_verif
7039,3571,2022-04-29,CROISSANT,1,10.0,25.5,10.0
7047,3575,2022-04-29,CROISSANT,1,10.0,28.9,10.0
7054,3582,2022-04-29,CROISSANT,1,10.0,25.5,10.0
7072,3599,2022-04-30,CROISSANT,1,10.0,19.5,10.0
7074,3600,2022-04-30,CROISSANT IFOOD,1,12.0,51.0,12.0


### Criando um arquivo HTML dos dados divergentes usando Plotly ("ts2" dataframe)

In [198]:
fig_divergentes = go.Figure(data=[go.Table(
    header=dict(values=list(ts2.columns),
                fill_color='seagreen',
                align='left'),
    cells=dict(values=[ts2['venda'], ts2['data'], ts2['produto'], ts2['quantidade'], ts2['valor_unit'], ts2['total_item'], ts2['total_item_verif']],
               fill_color='lightcyan',
               align='left'))
])


fig_divergentes.update_layout(
    title={
        'text': "Dados divergentes - Valores de venda - Ano 2022",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

py.plot(fig_divergentes, filename='results\Dados_divergentes_vendas.html')

'results\\Dados_divergentes_vendas.html'

In [100]:
# drop the 'confirm" and "total_item_verif" column

transactions = transactions.drop(['confirm'], axis=1)
transactions = transactions.drop(['total_item_verif'], axis=1)
transactions = transactions.drop (['diference'], axis=1)

> # **Exploração dos Dados**

In [101]:
# Find Unique Values of each item sold in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
# Pandas: get first 10 elements of a series
top_produtos = transactions.produto.value_counts()
print (top_produtos[:10])

CROISSANT                       2393
PASTEL DE NATA 60G              1339
CROISSANT  AMENDOAS 100G         742
FOLHADO MACA 80G                 719
SANDUICHE DE PEPERONI BRIE       596
COOKIES 80G                      559
CROISSANT IFOOD                  534
SOURDOUGH AZEITONA E ALECRIM     517
FOLHADO FRANGO 160G              500
BOMBOLONI DOCE DE LEITE 80G      487
Name: produto, dtype: int64


### Converting a Series to a dataframe and changing the namee of the column

In [170]:
top_produtos_df = top_produtos.to_frame()


In [171]:
top_produtos_df = top_produtos_df.reset_index()

In [174]:
top_produtos_df.rename(columns = {'produto':'quantidade', 'index':'produto'}, inplace = True)


In [176]:
top_produtos_df.head()

,produto,quantidade
0,CROISSANT,2393
1,PASTEL DE NATA 60G,1339
2,CROISSANT AMENDOAS 100G,742
3,FOLHADO MACA 80G,719
4,SANDUICHE DE PEPERONI BRIE,596


### Criando um arquivo HTML usando Plotly

In [200]:
fig_top_produtos = go.Figure(data=[go.Table(
    header=dict(values=list(top_produtos_df.columns),
                fill_color='seagreen',
                align='left'),
    cells=dict(values=[top_produtos_df['produto'], top_produtos_df['quantidade'] ],
               fill_color='lightcyan',
               align='left'))
])


fig_top_produtos.update_layout(
    title={
        'text': "Produtos mais vendidos - Ano 2022",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

py.plot(fig_top_produtos, filename='results\\Top_products_2022.html')

'results\\Top_products_2022.html'

### Find the percentage of itens sold

In [188]:
# find the percentage each item is selling
top_produtos_perc = transactions.produto.value_counts(True)
print (top_produtos_perc[:10])

CROISSANT                       0.092067
PASTEL DE NATA 60G              0.051516
CROISSANT  AMENDOAS 100G        0.028547
FOLHADO MACA 80G                0.027662
SANDUICHE DE PEPERONI BRIE      0.022930
COOKIES 80G                     0.021507
CROISSANT IFOOD                 0.020545
SOURDOUGH AZEITONA E ALECRIM    0.019891
FOLHADO FRANGO 160G             0.019237
BOMBOLONI DOCE DE LEITE 80G     0.018737
Name: produto, dtype: float64


#### Converting to a Dataframe

In [189]:
top_produtos_perc_df = top_produtos_perc.to_frame().reset_index()


In [190]:
top_produtos_perc_df.rename(columns = {'produto':'proporção', 'index':'produto'}, inplace = True)


In [191]:
top_produtos_perc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   produto    212 non-null    object 
 1   proporção  212 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.4+ KB


#### Criando um arquivo html usando Plotly

In [196]:
fig_top_produtos_perc = go.Figure(data=[go.Table(
    header=dict(values=list(top_produtos_perc_df.columns),
                fill_color='seagreen',
                align='left'),
    cells=dict(values=[top_produtos_perc_df['produto'], top_produtos_perc_df['proporção'] ],
               fill_color='lightcyan',
               align='left'))
])


fig_top_produtos_perc.update_layout(
    title={
        'text': "Produtos mais vendidos proporcional - Ano 2022",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

py.plot(fig_top_produtos_perc, filename='results\\top_products_proporc_2022.html')

'results\\top_products_proporc_2022.html'

## Algumas métricas importantes

In [ ]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
num_itens_vend = transactions.produto.unique().size

print('O número de diferentes itens vendido pela loja é:', num_itens_vend)

O número de diferentes itens vendido pela loja é: 212


In [ ]:
# Look at the average number of items purchased...

avg_items = transactions.groupby(by='venda')['produto'].count().mean()

print('A média de itens por venda é: %.2f itens' %avg_items)

A média de itens por venda é: 2.79 itens


In [ ]:
# ...and how that varies by time of day

single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['hora','minuto','hora_do_dia','data', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens vendidos por hora do dia é: \n{0}'.format(
    single_transaction.groupby(by='hora_do_dia')['item_count'].mean()))

O número médio de itens vendidos por hora do dia é: 
hora_do_dia
manhã    3.046419
noite    2.663510
tarde    2.783376
Name: item_count, dtype: float64


In [ ]:
# Find the frequency of each item purchased and by time of day

item_frequencies_all = transactions.produto.value_counts()
item_frequencies_morning = transactions[transactions.hora_do_dia == 'manhã'].produto.value_counts()
item_frequencies_afternoon = transactions[transactions.hora_do_dia == 'tarde'].produto.value_counts()
item_frequencies_evening = transactions[transactions.hora_do_dia == 'noite'].produto.value_counts()

In [ ]:
# Plot the frequency of purchase for each item

# https://community.plotly.com/t/errors-accessing-plotly-api/33271
# ok, so I found a solution to my problem - when I let my program run through, the error message produced is:
#chart_studio.exceptions.PlotlyRequestError: Authentication credentials were not provided.
#I Googled this error and there were suggestions to run plotly in offline mode by replacing:
#import chart_studio.plotly as py
#with



# Solving the problem of the 'plotly' online library with chart_studio
# https://plotly.com/python/getting-started-with-chart-studio/


button_layer_1_height = 1.12
button_layer_2_height = 1.065

item_threshold_all = 100
item_threshold_other = 20

trace_all = go.Bar(
            x=item_frequencies_all[item_frequencies_all > item_threshold_all].index,
            y=item_frequencies_all[item_frequencies_all > item_threshold_all].values,
            visible=True,
            name='Todo dia'
    )

trace_morning = go.Bar(
            x=item_frequencies_morning[item_frequencies_morning > item_threshold_other].index,
            y=item_frequencies_morning[item_frequencies_morning > item_threshold_other].values,
            visible=False,
            name='Manhã'
    )

trace_afternoon = go.Bar(
            x=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].index,
            y=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].values,
            visible=False,
            name='Tarde'
    )

trace_evening = go.Bar(
            x=item_frequencies_evening[item_frequencies_evening > 10].index,
            y=item_frequencies_evening[item_frequencies_evening > 10].values,
            visible=False,
            name='Noite'
    )

data = [trace_all, trace_morning, trace_afternoon, trace_evening]


In [ ]:

updatemenus = list([
    
    dict(
         buttons=list([   
            dict(label = 'Todo dia',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         {'title': 'Todo Dia'}]),
             
            dict(label = 'Manhã',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Manhã'}]),
             
            dict(label = 'Tarde',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Tarde'}]),
             
            dict(label = 'Noite',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Noite'}])
        ]),
        type='buttons',
        direction = 'right',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.3,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top' )
])
    

In [ ]:
            
layout = dict(title='Vendas Farina por hora do dia', showlegend=False,
              updatemenus=updatemenus)




In [ ]:


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Frequencia_vendas_hora_dia.html')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## Creating a plot to show the top 20 itens sold by month (fevereiro 2022 -outubro 2022)

In [ ]:
f_filter = (transactions['data']!='2022-06-23')#&(transactions['data']!='2022-07-30')
transactions[f_filter]

transactions_filtrada = transactions[f_filter]

### Criar a váriavel mes baseada na variavel horario

In [ ]:
transactions_filtrada['mes']=(transactions_filtrada['data'].dt.month)


## Doing the longer code way

In [ ]:
# Find the frequency of each item purchased by month

item_frequencies_periodo = transactions_filtrada.produto.value_counts()
item_frequencies_jan = transactions_filtrada[transactions_filtrada.mes== 1].produto.value_counts()
item_frequencies_fev = transactions_filtrada[transactions_filtrada.mes== 2].produto.value_counts()
item_frequencies_mar = transactions_filtrada[transactions_filtrada.mes== 3].produto.value_counts()
item_frequencies_abr = transactions_filtrada[transactions_filtrada.mes== 4].produto.value_counts()
item_frequencies_mai = transactions_filtrada[transactions_filtrada.mes== 5].produto.value_counts()
item_frequencies_jun = transactions_filtrada[transactions_filtrada.mes== 6].produto.value_counts()
item_frequencies_jul = transactions_filtrada[transactions_filtrada.mes== 7].produto.value_counts()
item_frequencies_ago = transactions_filtrada[transactions_filtrada.mes== 8].produto.value_counts()
item_frequencies_set = transactions_filtrada[transactions_filtrada.mes== 9].produto.value_counts()
item_frequencies_out = transactions_filtrada[transactions_filtrada.mes== 10].produto.value_counts()
item_frequencies_nov = transactions_filtrada[transactions_filtrada.mes== 11].produto.value_counts()
item_frequencies_dez = transactions_filtrada[transactions_filtrada.mes== 12].produto.value_counts()


In [ ]:
# Plot the frequency of purchase for each item by month

button_layer_1_height = 1.12
button_layer_2_height = 1.065

item_threshold_all = 250
item_threshold_other = 40

trace_periodo = go.Bar(
            x=item_frequencies_periodo[item_frequencies_periodo > item_threshold_all].index,
            y=item_frequencies_periodo[item_frequencies_periodo > item_threshold_all].values, text=item_frequencies_periodo,
            visible=True,
            name='Todo periodo'
    )

trace_jan = go.Bar(
            x=item_frequencies_jan[item_frequencies_jan > item_threshold_other].index,
            y=item_frequencies_jan[item_frequencies_jan > item_threshold_other].values,
            visible=False,
            name='Janeiro'
    )

trace_fev = go.Bar(
            x=item_frequencies_fev[item_frequencies_fev > item_threshold_other].index,
            y=item_frequencies_fev[item_frequencies_fev > item_threshold_other].values,
            visible=False,
            name='Fevereiro'
    )

trace_mar = go.Bar(
            x=item_frequencies_mar[item_frequencies_mar > item_threshold_other].index,
            y=item_frequencies_mar[item_frequencies_mar > item_threshold_other].values,
            visible=False,
            name='Março'
    )
trace_abr = go.Bar(
            x=item_frequencies_abr[item_frequencies_abr > item_threshold_other].index,
            y=item_frequencies_abr[item_frequencies_abr > item_threshold_other].values,
            visible=False,
            name='Abril'
)
trace_mai = go.Bar(
            x=item_frequencies_mai[item_frequencies_mai > item_threshold_other].index,
            y=item_frequencies_mai[item_frequencies_mai > item_threshold_other].values,
            visible=False,
            name='Maio'
)
trace_jun = go.Bar(
            x=item_frequencies_jun[item_frequencies_jun > item_threshold_other].index,
            y=item_frequencies_jun[item_frequencies_jun > item_threshold_other].values,
            visible=False,
            name='Junho'
)
trace_jul = go.Bar(
            x=item_frequencies_jul[item_frequencies_jul > 25].index,
            y=item_frequencies_jul[item_frequencies_jul > 25].values,
            visible=False,
            name='Julho'
)
trace_ago = go.Bar(
            x=item_frequencies_ago[item_frequencies_ago > item_threshold_other].index,
            y=item_frequencies_ago[item_frequencies_ago > item_threshold_other].values,
            visible=False,
            name='Agosto'
)
trace_set = go.Bar(
            x=item_frequencies_set[item_frequencies_set > item_threshold_other].index,
            y=item_frequencies_set[item_frequencies_set > item_threshold_other].values,
            visible=False,
            name='Setembro'
)
trace_out = go.Bar(
            x=item_frequencies_out[item_frequencies_out > item_threshold_other].index,
            y=item_frequencies_out[item_frequencies_out > item_threshold_other].values,
            visible=False,
            name='Outubro'
)
trace_nov = go.Bar(
            x=item_frequencies_nov[item_frequencies_nov > item_threshold_other].index,
            y=item_frequencies_nov[item_frequencies_nov > item_threshold_other].values,
            visible=False,
            name='Novembro'
)
trace_dez = go.Bar(
            x=item_frequencies_dez[item_frequencies_dez > item_threshold_other].index,
            y=item_frequencies_dez[item_frequencies_dez > item_threshold_other].values,
            visible=False,
            name='Dezembro'
)




data = [trace_periodo, trace_jan, trace_fev, trace_mar, trace_abr, trace_mai, trace_jun, trace_jul, trace_ago, trace_set, trace_out, trace_nov, trace_dez]


In [ ]:

updatemenus2 = list([

    dict
        (
         buttons=list([   
            
                dict(label = 'Todo Periodo',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False]},
                         {'title': 'Todo período'}]),
             
                dict(label = 'Janeiro',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False]},
                         {'title': 'Janeiro'}]),
             
                dict(label = 'Fevereiro',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False]},
                         {'title': 'Fevereiro'}]),
             
                dict(label = 'Março',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False]},
                         {'title': 'Março'}]),
            
                dict(label = 'Abril',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False]},
                         {'title': 'Abril'}]),

                dict(label = 'Maio',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False]},
                         {'title': 'Maio'}]),

                dict(label = 'Junho',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False]},
                         {'title': 'Junho'}]),

                dict(label = 'Julho',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False]},
                         {'title': 'Julho'}]),
        
                dict(label = 'Agosto',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False]},
                         {'title': 'Agosto'}]),

                dict(label = 'Setembro',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False]},
                         {'title': 'Setembro'}]),

                dict(label = 'Outubro',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False]},
                         {'title': 'Outubro'}]),

                dict(label = 'Novembro',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False]},
                         {'title': 'Novembro'}]),
                        
                dict(label = 'Dezembro',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True]},
                         {'title': 'Dezembro'}])
         ]),
        type='buttons',
        direction = 'right',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.3,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top' 
        )
])
                


In [ ]:

layout = dict(title='Vendas Farina por item - ano 2022', showlegend=False, updatemenus=updatemenus2)



In [ ]:



fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Frequencia_vendas_itens_mensal.html')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



# Creating an animated frequencie line 

In [ ]:
# Sales trend by date
# https://www.youtube.com/watch?v=Xjm7fOw6gjY
# use reset index for transforming the data in a variable not a index

grouped_by_date = pd.DataFrame(transactions.groupby(by='horario')['item_count'].sum().reset_index())

In [ ]:
grouped_by_date.head()

,horario,item_count
0,2022-02-01,74
1,2022-02-02,88
2,2022-02-03,81
3,2022-02-04,110
4,2022-02-05,112


In [ ]:
import plotly.figure_factory as ff

# Create a plot.ly-friendly table for Sales per day

table = ff.create_table(grouped_by_date)
py.plot(table, filename='tabela_vendas_por_dia_farina.html')


'tabela_vendas_por_dia_farina.html'

In [ ]:
vendas = px.line(grouped_by_date, x = 'horario', y = 'item_count', title = 'Itens vendidos por dia', labels= {'item_count' : 'Quantidade de itens vendidos','horario': 'Data'})
py.plot(vendas, filename='Vendas_por_dia_linha.html')

'Vendas_por_dia_linha.html'

In [ ]:
# ajustando o gráfio para retirar dois outliers (490 em 2022-07-30 , 2303 em 2022-06-23 ), utilizando-se de filtro com a função 'where'
indice_atip = np.where((grouped_by_date['item_count'] < 400))
linha=grouped_by_date.loc[indice_atip]

In [ ]:
# https://towardsdatascience.com/line-chart-animation-with-plotly-on-jupyter-e19c738dc882



#linha=grouped_by_date: usar se precisar demonstrar os dois outliers

trace1 = go.Scatter(x=linha['horario'][:2],
                    y=linha['item_count'][:2],
                    mode='lines',
                    line=dict(width=1.5),
                    fill = 'tozeroy',
                   fillcolor = 'lightblue'
)
frames = [dict(data= [dict(type='scatter',
                           x=linha['horario'][:k+1],
                           y=linha['item_count'][:k+1]),
                     ],
               traces= [0],  
              )for k  in  range(1, len(linha)-1)]
layout = go.Layout(width=1180,
                   height=750,
                   showlegend=False,
                   hovermode='closest',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.06,
                            x=0.12,
                            xanchor='center',
                            yanchor='auto',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=30, 
                                                  redraw=False),
                                                  transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        
                    ]              
                  )
layout.update(xaxis =dict(range=['2022-02-01', '2022-10-31'], autorange=False),
              yaxis =dict(range=[0, 250], autorange=False)); # ajustar o range se precisar colocar os outliers
fig_vendas = go.Figure(data=[trace1], frames=frames, layout=layout)

py.plot(fig_vendas, filename='Vendas_por_dia_farina_interativo.html')

'Vendas_por_dia_farina_interativo.html'

# Investigando dados atipicos

In [ ]:
# Vendas atípicas no dia 23 de Junho de 2022 entre 16:00 e 17:00

df_invest = pd.DataFrame(transactions).query("data == 'Jun 23, 2022'").groupby(by='hora')['item_count'].sum().reset_index()


In [ ]:
dataframe_to_pdf(df_invest, "investiga_dados_atipicos.pdf", numpages=(1, 1), pagesize=(11.7, 8.3))

In [ ]:
graf = px.line(df_invest, x = 'hora', y = 'item_count', labels= {'item_count':'Itens Vendidos'}, log_y= True,
title= 'Investigando dados atipicos - qantidade itens vendidos no dia 23 de Junho 2022')

py.plot(graf, filename='investiga_graf.html')


'investiga_graf.html'

### Encontrados dados atípicos nos dias 23 de Junho de 2022 e 30 de Julho de 2022, portanto uma filtragem desses dados se faz necessária

In [ ]:
f_filter = (transactions['data']!='2022-06-23')&(transactions['data']!='2022-07-30')
transactions[f_filter]

transactions_filtrada = transactions[f_filter]

# Vendas dias de semana (DDS) vs Final de semana (FDS)

In [ ]:
dds_vendas2= pd.DataFrame(transactions_filtrada[transactions_filtrada.tipo_de_dia == 'DDS'].groupby(by='horario')['item_count'].sum())
fds_vendas2= pd.DataFrame(transactions_filtrada[transactions_filtrada.tipo_de_dia == 'FDS'].groupby(by='horario')['item_count'].sum())

### Vendas dia de semana(DDS)

In [ ]:
print('O menor número de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas2.item_count.min()))
print('O maior número de vendas em um dia de semana é: \n{0}'.format(
    dds_vendas2.item_count.max()))
print('O número médio de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas2.item_count.mean()))

O menor número de vendas em um dia de semana foi: 
1
O maior número de vendas em um dia de semana é: 
241
O número médio de vendas em um dia de semana foi: 
75.81005586592178


### Vendas final de semana (FDS)

In [ ]:
print('O menor número de vendas em um dia de fim-de-semana foi: \n{0}'.format(
    fds_vendas2.item_count.min()))
print('O  maior número de vendas em um dia fim-de-semana é: \n{0}'.format(
    fds_vendas2.item_count.max()))
print('O número médio de vendas em um dia fim-de-semana foi: \n{0}'.format(
    fds_vendas2.item_count.mean()))

O menor número de vendas em um dia de fim-de-semana foi: 
65
O  maior número de vendas em um dia fim-de-semana é: 
227
O número médio de vendas em um dia fim-de-semana foi: 
133.73611111111111


### Número de itens médios vendidos por cada pedido

In [ ]:
single_transaction = pd.DataFrame(transactions_filtrada.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions_filtrada[['tipo_de_dia','horario', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens em cada venda por tipo de dia é: \n{0}'.format(
    single_transaction.groupby(by='tipo_de_dia')['item_count'].mean()))

O número médio de itens em cada venda por tipo de dia é: 
tipo_de_dia
DDS    2.650391
FDS    2.836230
Name: item_count, dtype: float64


## Itens vendidos por tipo de dia

In [ ]:
# Histogram of items sold by type of day

trace_dds = go.Histogram(
    x=dds_vendas2.item_count,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_vendas2.item_count,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição das vendas por dias de semana/ finais de semana',
                   xaxis=dict(title='Número de Itens Vendidos'),
                   yaxis=dict(title='Frequencia de vendas'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='tipo-de_dia_itens_histograma.html')

'tipo-de_dia_itens_histograma.html'

# Faturamento por tipo de dia (DDS x FDS)

In [ ]:
dds_fatu= pd.DataFrame(transactions_filtrada[transactions_filtrada.tipo_de_dia == 'DDS'].groupby(by='horario')['total_item'].sum())
fds_fatu= pd.DataFrame(transactions_filtrada[transactions_filtrada.tipo_de_dia == 'FDS'].groupby(by='horario')['total_item'].sum())

In [ ]:
# Histograma

trace_dds = go.Histogram(
    x=dds_fatu[dds_fatu.total_item  < 7900].total_item,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_fatu[fds_fatu.total_item < 7900].total_item,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição do faturamento por dias de semana/ finais de semana',
                   xaxis=dict(title='Faturamento por dia'),
                   yaxis=dict(title='Frequencia de faturamento diário'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='tipo-de_dia_faturamento_histograma.html')

'tipo-de_dia_faturamento_histograma.html'

# Número de itens vendidos por dia da semana

In [ ]:
grouped_by_date_filtrada = pd.DataFrame(transactions_filtrada.groupby(by='horario')['item_count'].sum())

In [ ]:
# Find the number of itens sold per day of week

agregado_vendas_por_dia = grouped_by_date_filtrada.merge(transactions[['dia_da_semana', 'horario']], left_index=True, right_on='horario',
        how= 'left', validate='1:m')



In [ ]:
agregado_vendas_por_dia = agregado_vendas_por_dia[~agregado_vendas_por_dia.index.duplicated(keep='first')]

dias = ['segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira','sábado','domingo']

vendas_por_dia = []

for dia in dias:
    
    df_dias= agregado_vendas_por_dia[agregado_vendas_por_dia.dia_da_semana == dia]
    
    vendas_por_dia.append(list(df_dias['item_count']))
    


In [ ]:
traces = []

for i, j in enumerate(dias):
    
    trace = go.Box(
    y=vendas_por_dia[i],
    name = j
    )
    traces.append(trace)

data = traces

layout = go.Layout(title='Vendas por dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'))

fig = go.Figure(data = data, layout = layout)

py.plot(fig, filename='Vendas_por_dia_da_semana_Boxplot.html')

'Vendas_por_dia_da_semana_Boxplot.html'

# Horário de vendas dos itens por dia da semana

In [ ]:
dia_hora_df = transactions_filtrada[[ 'hora_minuto','dia_da_semana', 'hora', 'item_count']]


### Modo Simples

In [ ]:
fig1 = px.box(dia_hora_df, y = 'hora', hover_data=['item_count'], title= ' Vendas agregadas por hora')

py.iplot(fig1, filename = 'Boxplot_Vendas_agregadas_hora.html')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
fig = px.box(dia_hora_df, y = 'hora', x = 'dia_da_semana', color='dia_da_semana', labels={'hora': 'Hora','dia_da_semana':'Dia da Semana'},
title="Hora das vendas dia da semana", hover_data = [dia_hora_df.hora_minuto], points="all", 
category_orders= {"dia_da_semana": ['sábado','domingo','segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira']})

import plotly
py.plot(fig, filename='Vendas_agregadas_Hora_dias_semana.html')

'Vendas_agregadas_Hora_dias_semana.html'

In [ ]:
# https://stackoverflow.com/questions/59815797/how-to-save-plotly-express-plot-into-a-html-or-static-image-file
# fig.write_html(r"C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\Vendas_por_dia_Boxplot_simples2.html")


### Modo Complexo - Graphic Objects

### How to use Graphics Objects


In [ ]:
#  https://www.youtube.com/watch?v=AwtQ7vJg0-o

In [ ]:
dom = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'domingo']
seg = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'segunda-feira']
ter = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'terça-feira']
qua = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'quarta-feira']
qui = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'quinta-feira']
sex = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'sexta-feira']
sab = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'sábado']

In [ ]:
trace0 = go.Box(
    y= dom, boxpoints= "outliers", jitter = 0.5,
     name = 'Domingo', whiskerwidth= 1, notchwidth=0.5
)

trace1 = go.Box(
    y= seg,  boxpoints = 'outliers', jitter = 0.5,
    name = 'Segunda',  whiskerwidth= 1, notchwidth=0.5
)

trace2 = go.Box(
    y= ter, boxpoints = 'outliers', jitter = 0.5,
    name = 'Terça', whiskerwidth= 1, notchwidth=0.5
)

trace3 = go.Box(
    y= qua, boxpoints = 'outliers', jitter = 0.5,
    name = 'Quarta', whiskerwidth= 1, notchwidth=0.5
)

trace4 = go.Box(
    y= qui,boxpoints = 'outliers', jitter = 0.5,
    name = 'Quinta', whiskerwidth= 1, notchwidth=0.5
)

trace5 = go.Box(
    y= sex,boxpoints = 'outliers', jitter = 0.5,
    name = 'Sexta', whiskerwidth= 1, notchwidth=0.5
)

trace6 = go.Box(
    y= sab,boxpoints = 'outliers', jitter = 0.5,
    name = 'Sábado', whiskerwidth= 1, notchwidth=0.5
)

In [ ]:
data = [trace6, trace0, trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(title= 'Vendas por hora e dia da semana')

In [ ]:
fig = go.Figure(data = data, layout = layout)


In [ ]:
py.plot(fig, filename='Vendas_hora_por_dia_Boxplot_GO.html')

'Vendas_hora_por_dia_Boxplot_GO.html'

## Quantidade de itens vendidos por dia da semana e período


In [159]:
group_by_data_e_hora_do_dia = transactions_filtrada.groupby(by=['dia_da_semana','hora_do_dia'])['item_count'].sum()

NameError: name 'transactions_filtrada' is not defined

In [ ]:
# https://stackoverflow.com/questions/39275294/sort-by-certain-order-situation-pandas-dataframe-groupby
# #### Alterando a ordem de apresentação dos dias da semana(Seg - Dom), usando a lista 'dias' anteriormente criada

group_by_data_e_hora_do_dia = group_by_data_e_hora_do_dia.reindex(dias, level= 'dia_da_semana')

In [ ]:
group_by_data_e_hora_do_dia = pd.DataFrame(group_by_data_e_hora_do_dia).reset_index()

In [ ]:
group_by_data_e_hora_do_dia

,dia_da_semana,hora_do_dia,item_count
0,segunda-feira,manhã,587
1,segunda-feira,noite,1354
2,segunda-feira,tarde,548
3,terça-feira,manhã,545
4,terça-feira,noite,1359
5,terça-feira,tarde,619
6,quarta-feira,manhã,642
7,quarta-feira,noite,1403
8,quarta-feira,tarde,811
9,quinta-feira,manhã,526


NameError: name 'group_by_data_e_hora_do_dia' is not defined

In [ ]:
group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count

0      587
3      545
6      642
9      526
12     744
15    1512
18    2310
Name: item_count, dtype: int64

In [ ]:
trace_manhã = go.Bar(
    y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].dia_da_semana,
    name = "manhã",
    opacity=0.6,
)

trace_tarde = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].dia_da_semana,
    name = "tarde",
    opacity=0.6
)

trace_noite = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].dia_da_semana,
    name = "noite",
    opacity=0.6
)

layout = go.Layout(title='Vendas por período e dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'), hovermode= 'x')

data = [trace_manhã, trace_tarde, trace_noite]

fig = go.Figure(data = data, layout = layout)  

py.iplot(fig, filename='vendas-por-dia-periodo')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



### Vendas por hora

In [ ]:
vendas_por_hora = pd.DataFrame(
                transactions_filtrada[(transactions_filtrada.hora > 1) & 
                (transactions_filtrada.hora < 21)].groupby(by='hora')['item_count'].sum())

In [ ]:
vendas_por_hora

,item_count
hora,
6,41
7,524
8,1289
9,1749
10,1817
11,1446
12,737
13,627
14,769


In [ ]:
trace = go.Scatter(x=list(vendas_por_hora.index),
                   y=list(vendas_por_hora.item_count),
                   fill = 'tozeroy',
                   fillcolor = 'lightblue')

data = [trace]

layout = dict(
    title='Total de volume de vendas por hora',
    yaxis = dict( title = 'Volume de Itens Vendidos'),
    xaxis=dict( title = 'Hora',
        rangeselector=dict(
            buttons=list([
                dict(count = 12,
                     label='Manhã',
                     step='hour',
                     stepmode='backward'),
                dict(count = 8,
                     label='Tarde',
                     step='hour',
                     stepmode='backward'),
                dict(count = 4,
                    label='Noite',
                    step='hour',
                    stepmode='backward'),
                dict(step='all')
            ]),
            visible = True
        ),
        rangeslider=dict(
            visible = True
        )
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'vendas-por-hora')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



# Apriori Algorithm, Method 1

In [ ]:
# Create a list of lists that is conducive to the format expected from apyori

list_of_lists = []

for transaction in list(set(transactions_filtrada.venda)):
    
    df = transactions_filtrada[transactions_filtrada.venda == transaction]
    values = df.produto.values
    
    list_of_lists.append(list(values))

In [ ]:
# imprting the 'apriori' function from the 'apyori' module
from apyori import apriori

# Create a list of association rules
association_rules = apriori(list_of_lists, min_confidence = 0.2, min_support = 0.005,min_lift = 2, min_length=2)  
association_results = list(association_rules)

In [ ]:
# print(association_results)

In [ ]:
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")


Rule: ADICIONAL DE MUSSARELA -> ADICIONAL DE PRESUNTO
Support: 0.0061068702290076335
Confidence: 0.5416666666666666
Lift: 70.95833333333333
Rule: CROISSANT -> ADICIONAL DE MUSSARELA
Support: 0.009395184967704051
Confidence: 0.8333333333333333
Lift: 4.038607474862455
Rule: CROISSANT -> ADICIONAL DE PRESUNTO
Support: 0.006928948913681738
Confidence: 0.9076923076923077
Lift: 4.398975526465566
Rule: CROISSANT -> AGUA MINERAL S/GAS 500ML
Support: 0.01996476805637111
Confidence: 0.4197530864197531
Lift: 2.0342615428936814
Rule: BOWL DE IOGURTE FRUTAS AVEIA -> PAO SOURDOUGH OVOS
Support: 0.007868467410452143
Confidence: 0.24814814814814815
Lift: 9.107678799489145
Rule: CROISSANT -> CAFE CATUAI VERMELHO C/LEITE G
Support: 0.007868467410452143
Confidence: 0.43506493506493504
Lift: 2.1084677985645546
Rule: CROISSANT -> CAFE CATUAI VERMELHO G 200ML
Support: 0.013740458015267175
Confidence: 0.46799999999999997
Lift: 2.2680819578827545
Rule: CROISSANT -> CAFE COLMEIA C/ LEITE G 200ML
Support: 0.005

## Apriori Algorithm, Method 1 - Manhã

In [ ]:
manha_filter = (transactions_filtrada['hora_do_dia']=='manhã')
transactions_filtrada_manha = transactions_filtrada[manha_filter]
# Create a list of lists that is conducive to the format expected from apyori


list_of_lists_manha = []

for transaction in list(set(transactions_filtrada_manha.venda)):
    
    df = transactions_filtrada_manha[transactions_filtrada_manha.venda == transaction]
    values = df.produto.values
    
    list_of_lists_manha.append(list(values))

In [ ]:
# imprting the 'apriori' function from the 'apyori' module
from apyori import apriori

# Create a list of association rules
association_rules_manha = apriori(list_of_lists_manha, min_confidence = 0.2, min_support = 0.005,min_lift = 3, min_length=3)  
association_results_manha = list(association_rules_manha)

In [ ]:
# association_results_manha

In [ ]:
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: ADICIONAL DE MUSSARELA -> ADICIONAL DE PRESUNTO
Support: 0.009764758100310697
Confidence: 0.5945945945945946
Lift: 40.5945945945946
Rule: ADICIONAL DE MUSSARELA -> AGUA MINERAL S/GAS 500ML
Support: 0.005326231691078562
Confidence: 0.3243243243243244
Lift: 5.294947121034079
Rule: CROISSANT -> ADICIONAL DE MUSSARELA
Support: 0.015090989791389259
Confidence: 0.918918918918919
Lift: 3.344627341396324
Rule: CROISSANT -> ADICIONAL DE PRESUNTO
Support: 0.013315579227696404
Confidence: 0.9090909090909091
Lift: 3.3088559259803203
Rule: CROISSANT BRIE PARMA MEL DE -> AGUA MINERAL SEM GAS 500 ML
Support: 0.007545494895694629
Confidence: 0.2
Lift: 5.006666666666667
Rule: BOWL DE IOGURTE FRUTAS AVEIA -> PAO SOURDOUGH OVOS
Support: 0.024411895250776743
Confidence: 0.2972972972972973
Lift: 3.9170222854433385
Rule: SELECAO DE PAES GELEIA -> BOWL DE IOGURTE FRUTAS AVEIA
Support: 0.010208610741233911
Confidence: 0.2674418604651163
Lift: 3.2570081709616594
Rule: CAFE CATUAI VERMELHO C/LEITE G -> LA

In [ ]:
rule_list_manha = []
for item in association_results_manha:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]

    rule_list_manha.append ( items[0] + " -> " + items[1])

support_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    #items = [x for x in pair]
    

    #second index of the inner list
    support_list_manha.append( str(item[1]))

confidence_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    

    #second index of the inner list
    confidence_list_manha.append(str(item[2][0][2]))

lift_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    
    #second index of the inner list
    lift_list_manha.append(str(item[2][0][3]))

In [ ]:
association_results_manha_df=pd.DataFrame(list
(zip(rule_list_manha, support_list_manha, confidence_list_manha, lift_list_manha)), columns=['Rule','Support', 'Confidence', 'Lift'])

In [ ]:
association_results_manha_df.sort_values(by=['Support'],ascending=False)

,Rule,Support,Confidence,Lift
5,BOWL DE IOGURTE FRUTAS AVEIA -> PAO SOURDOUGH ...,0.024411895250776743,0.2972972972972973,3.9170222854433385
2,CROISSANT -> ADICIONAL DE MUSSARELA,0.015090989791389259,0.918918918918919,3.344627341396324
3,CROISSANT -> ADICIONAL DE PRESUNTO,0.013315579227696404,0.9090909090909091,3.3088559259803203
6,SELECAO DE PAES GELEIA -> BOWL DE IOGURTE FRUT...,0.010208610741233911,0.2674418604651163,3.2570081709616594
0,ADICIONAL DE MUSSARELA -> ADICIONAL DE PRESUNTO,0.009764758100310697,0.5945945945945946,40.5945945945946
16,CROISSANT -> ADICIONAL DE MUSSARELA,0.009320905459387484,0.5675675675675677,42.624324324324334
10,CROISSANT BRIE PARMA MEL DE -> SUCO DE LARANJA...,0.00887705281846427,0.22222222222222224,6.032128514056224
14,SUCO DE LARANJA 400ML -> CROISSANT OVOS BACON MEL,0.008433200177541057,0.2289156626506024,5.157469879518072
12,SOURDOUGH 369G IFOOD -> CROISSANT IFOOD,0.008433200177541057,0.59375,7.309938524590163
11,CROISSANT IFOOD -> PASTEL DE NATA 60G IFOOD,0.008433200177541057,0.5428571428571428,6.683372365339578


# All day results

In [ ]:
rule_list = []
for item in association_results:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]

    rule_list.append ( items[0] + " -> " + items[1])

support_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    #items = [x for x in pair]
    

    #second index of the inner list
    support_list.append( str(item[1]))
confidence_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    

    #second index of the inner list
    confidence_list.append(str(item[2][0][2]))
lift_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    
    #second index of the inner list
    lift_list.append(str(item[2][0][3]))

 ### Convert a list to a Dataframe

In [ ]:
association_results_df=pd.DataFrame(list
(zip(rule_list, support_list, confidence_list, lift_list)), columns=['Rule','Support', 'Confidence', 'Lift'])

In [ ]:
association_results_df

,Rule,Support,Confidence,Lift
0,ADICIONAL DE MUSSARELA -> ADICIONAL DE PRESUNTO,0.0061068702290076335,0.5416666666666666,70.95833333333333
1,CROISSANT -> ADICIONAL DE MUSSARELA,0.009395184967704051,0.8333333333333333,4.038607474862455
2,CROISSANT -> ADICIONAL DE PRESUNTO,0.006928948913681738,0.9076923076923077,4.398975526465566
3,CROISSANT -> AGUA MINERAL S/GAS 500ML,0.01996476805637111,0.4197530864197531,2.0342615428936814
4,BOWL DE IOGURTE FRUTAS AVEIA -> PAO SOURDOUGH ...,0.007868467410452143,0.24814814814814815,9.107678799489145
5,CROISSANT -> CAFE CATUAI VERMELHO C/LEITE G,0.007868467410452143,0.43506493506493504,2.1084677985645546
6,CROISSANT -> CAFE CATUAI VERMELHO G 200ML,0.013740458015267175,0.46799999999999997,2.2680819578827545
7,CROISSANT -> CAFE COLMEIA C/ LEITE G 200ML,0.00551967116852613,0.5280898876404494,2.5592973211487915
8,CROISSANT -> CAFE COLMEIA G 200ML,0.007633587786259542,0.4276315789473684,2.0724433094688917
9,CROISSANT -> LARANJA 400 ML,0.021374045801526718,0.6275862068965518,3.041489215551587


In [ ]:
table_2 = ff.create_table(association_results_df, height_constant=40)
py.plot(table_2, filename='Apriori_modelo_01.html')

'Apriori_modelo_01.html'

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(association_results_df.columns),
                fill_color='seagreen',
                align='left'),
    cells=dict(values=[association_results_df['Rule'], association_results_df['Support'], association_results_df['Confidence'], 
                       association_results_df['Lift']],
               fill_color='lightcyan',
               align='left'))
])




In [ ]:


fig.update_layout(
    title={
        'text': "Algoritimo Apriori Metódo 1",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


In [ ]:

py.plot(fig, filename='results\Apriori Method 01 table.html')

# Apriori Algorithm, Method 2

In [ ]:
def apriori_results_method_2(df = transactions_filtrada, min_support = 0.005, hora_do_dia = None, dia_da_semana = None,
                            min_threshold = 1, metric = 'lift'):
    
    """
    Takes in a data frame and returns the results of the Apriori algorithm using the mlxtend Python package.
    """
    
    from mlxtend.frequent_patterns import apriori
    from mlxtend.frequent_patterns import association_rules

    if hora_do_dia:
        
        df = df[df.hora_do_dia == hora_do_dia]
        
    if dia_da_semana:
        
        df = df[df.tipo_de_dia == dia_da_semana]
        
    dummies_df = df.groupby(['venda', 'produto'])['produto'].count().unstack().\
                 reset_index().fillna(0).set_index('venda')
    
    def encode_units(x):
    
        if x <= 0:
            return 0
        if x >= 1:
            return 1
    
    dummies_df = dummies_df.applymap(encode_units)
    
    frequent_itemsets = apriori(dummies_df, min_support= min_support, use_colnames=True)

    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)
    
    rules.drop_duplicates(subset='leverage', keep='first', inplace=True)
    rules = rules.sort_values(by='lift', ascending=False)
    
    return rules

In [ ]:
all_day_method_2 = apriori_results_method_2()

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
all_day_method_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 114 to 70
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         60 non-null     object 
 1   consequents         60 non-null     object 
 2   antecedent support  60 non-null     float64
 3   consequent support  60 non-null     float64
 4   support             60 non-null     float64
 5   confidence          60 non-null     float64
 6   lift                60 non-null     float64
 7   leverage            60 non-null     float64
 8   conviction          60 non-null     float64
dtypes: float64(7), object(2)
memory usage: 4.7+ KB


In [ ]:
fig_a = go.Figure(data=[go.Table(header=dict(values=list(all_day_method_2.columns), fill_color='seagreen', align='left'),
cells=dict(values=[all_day_method_2['antecedents'], 
                   all_day_method_2['consequents'], 
                   all_day_method_2['antecedent support'],
                   all_day_method_2['consequent support'],
                   all_day_method_2['support'],
                   all_day_method_2['confidence'], 
                   all_day_method_2['lift'], 
                   all_day_method_2['leverage'],
                   all_day_method_2['conviction'],],
fill_color='lightcyan', align='left'))])

# py.plot(fig, filename='Apriori Method 02 - all Day.html')

# fig.show()


In [ ]:
dataframe_to_pdf(all_day_method_2, "Apriori_modelo_02.pdf", numpages=(1, 2), pagesize=(11.7, 8.3))

In [ ]:
all_day_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
114,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009395,0.007634,0.005637,0.600000,78.600000,0.005565,2.480916
115,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.006929,0.011274,0.005637,0.813559,72.161017,0.005559,5.303166
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.011274,0.007634,0.006107,0.541667,70.958333,0.006021,2.165163
86,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.046389,0.011861,0.005167,0.111392,9.391152,0.004617,1.112008
30,(BOWL DE IOGURTE FRUTAS AVEIA),(PAO SOURDOUGH OVOS),0.031709,0.027246,0.007868,0.248148,9.107679,0.007005,1.293811
82,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.045449,0.024427,0.005989,0.131783,5.394864,0.004879,1.123650
96,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.046741,0.022901,0.005402,0.115578,5.046901,0.004332,1.104788
116,"(ADICIONAL DE MUSSARELA, ADICIONAL DE PRESUNTO)",(CROISSANT),0.006107,0.206342,0.005637,0.923077,4.473534,0.004377,10.317557
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.206342,0.007634,0.006929,0.033580,4.398976,0.005354,1.026848
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.206342,0.011274,0.009395,0.045532,4.038607,0.007069,1.035892


In [ ]:
morning_method_2 = apriori_results_method_2(hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
217,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.013316,0.016423,0.009321,0.700000,42.624324,0.009102,3.278592
216,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.015091,0.014647,0.009321,0.617647,42.168449,0.009100,2.577077
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.016423,0.014647,0.009765,0.594595,40.594595,0.009524,2.430537
132,(COOKIES 80G IFOOD),(CROISSANT IFOOD),0.009321,0.081225,0.005770,0.619048,7.621390,0.005013,2.411784
176,(SOURDOUGH 369G IFOOD),(CROISSANT IFOOD),0.014203,0.081225,0.008433,0.593750,7.309939,0.007280,2.261600
174,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.081225,0.015535,0.008433,0.103825,6.683372,0.007171,1.098519
178,(CROISSANT MUSSATRELA DE),(SUCO DE LARANJA 400ML),0.027075,0.036840,0.006658,0.245902,6.674896,0.005660,1.277234
170,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.039947,0.036840,0.008877,0.222222,6.032129,0.007405,1.238349
196,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.050599,0.024856,0.007102,0.140351,5.646617,0.005844,1.134351
246,"(CROISSANT, PAO SOURDOUGH OVOS)",(CAFE CATUAI VERMELHO G 200ML),0.017310,0.055038,0.005326,0.307692,5.590571,0.004374,1.364946


In [ ]:
afternoon_method_2 = apriori_results_method_2(hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
118,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.008975,0.007085,0.006613,0.736842,103.992982,0.006550,3.773075
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.010392,0.007085,0.007085,0.681818,96.227273,0.007012,3.120588
119,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.006613,0.010392,0.006613,1.000000,96.227273,0.006544,inf
84,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.045820,0.011809,0.005196,0.113402,9.602887,0.004655,1.114587
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.168162,0.007085,0.006613,0.039326,5.550187,0.005422,1.033560
32,(CAFE INTENSO CAPSULA P 50ML),(CAFE EQUILIBRADO CAPSULA P),0.030704,0.042985,0.007085,0.230769,5.368555,0.005766,1.244119
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.168162,0.010392,0.008975,0.053371,5.135725,0.007227,1.045402
96,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.053377,0.032121,0.008030,0.150442,4.683628,0.006316,1.139274
12,(BOMBOLONI DOCE DE LEITE 80G),(BOMBOLONI NUTELLA 80G),0.059046,0.048181,0.012754,0.216000,4.483059,0.009909,1.214054
64,(CROISSANT),(LARANJA 400 ML),0.168162,0.028342,0.017005,0.101124,3.567978,0.012239,1.080970


In [ ]:
evening_method_2 = apriori_results_method_2(hora_do_dia = 'noite')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
evening_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
92,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.055247,0.024367,0.006273,0.113537,4.659518,0.004926,1.100591
0,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188661,0.008926,0.006514,0.034527,3.867941,0.004830,1.026516
12,(CROISSANT BRIE PARMA MEL DE),(AGUA MINERAL SEM GAS 500 ML),0.055247,0.040290,0.007479,0.135371,3.359961,0.005253,1.109968
14,(CROISSANT MUSSATRELA DE),(AGUA MINERAL SEM GAS 500 ML),0.038601,0.040290,0.005066,0.131250,3.257672,0.003511,1.104703
74,(CROISSANT),(LARANJA 400 ML),0.188661,0.020024,0.011821,0.062660,3.129218,0.008044,1.045486
88,(CROISSANT MUSSATRELA DE),(CROISSANT BRIE PARMA MEL DE),0.038601,0.055247,0.006514,0.168750,3.054449,0.004381,1.136545
4,(AGUA MINERAL S/GAS 500ML),(BOMBOLONI NUTELLA 80G),0.042461,0.046080,0.005549,0.130682,2.836001,0.003592,1.097320
128,(PIZZA AL TAGLIO PEPPERONI 210G),(PIZZA AL TAGLIO GORGONZOLA E),0.034499,0.055247,0.005066,0.146853,2.658106,0.003160,1.107374
18,(BOMBOLONI DOCE DE LEITE 80G),(BOMBOLONI NUTELLA 80G),0.056936,0.046080,0.006755,0.118644,2.574763,0.004132,1.082333
50,(COCA COLA LATA 350 ML),(PIZZA AL TAGLIO GORGONZOLA E),0.049940,0.055247,0.006996,0.140097,2.535810,0.004237,1.098673


In [ ]:
weekend_method_2 = apriori_results_method_2(dia_da_semana = 'FDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
weekend_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
174,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009426,0.006480,0.005007,0.531250,81.981534,0.004946,2.119509
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.010604,0.006480,0.005007,0.472222,72.872475,0.004939,1.882459
130,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.064507,0.012077,0.006186,0.095890,7.940194,0.005407,1.092703
50,(BOWL DE IOGURTE FRUTAS AVEIA),(PAO SOURDOUGH OVOS),0.043004,0.032695,0.009426,0.219178,6.703690,0.008020,1.238829
124,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.047717,0.027982,0.007658,0.160494,5.735543,0.006323,1.157845
152,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.045066,0.022975,0.005302,0.117647,5.120664,0.004267,1.107295
134,(CROISSANT MUSSATRELA DE),(SUCO DE LARANJA 400ML),0.036230,0.027982,0.005007,0.138211,4.939238,0.003994,1.127907
68,(CAFE INTENSO CAPSULA P 50ML),(CAFE EQUILIBRADO CAPSULA P),0.022091,0.047717,0.005007,0.226667,4.750206,0.003953,1.231400
176,"(ADICIONAL DE MUSSARELA, ADICIONAL DE PRESUNTO)",(CROISSANT),0.005007,0.233284,0.005007,1.000000,4.286616,0.003839,inf
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.233284,0.006480,0.006480,0.027778,4.286616,0.004968,1.021906


In [ ]:
weekday_method_2 = apriori_results_method_2(dia_da_semana = 'DDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
weekday_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
102,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009375,0.008398,0.006055,0.645833,76.899225,0.005976,2.799816
103,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.007227,0.011719,0.006055,0.837838,71.495495,0.005970,6.094401
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.011719,0.008398,0.006836,0.583333,69.457364,0.006738,2.379844
26,(BOWL DE IOGURTE FRUTAS AVEIA),(PAO SOURDOUGH OVOS),0.024219,0.023633,0.006836,0.282258,11.943482,0.006264,1.360332
86,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.047852,0.022852,0.005469,0.114286,5.001221,0.004375,1.103232
104,"(ADICIONAL DE MUSSARELA, ADICIONAL DE PRESUNTO)",(CROISSANT),0.006836,0.188477,0.006055,0.885714,4.699334,0.004766,7.100830
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.188477,0.008398,0.007227,0.038342,4.565369,0.005644,1.031137
12,(CROISSANT BRIE PARMA MEL DE),(AGUA MINERAL SEM GAS 500 ML),0.043945,0.033594,0.006641,0.151111,4.498191,0.005164,1.138437
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188477,0.011719,0.009375,0.049741,4.244560,0.007166,1.040012
16,(BOMBOLONI DOCE DE LEITE 80G),(BOMBOLONI NUTELLA 80G),0.046484,0.041016,0.007227,0.155462,3.790316,0.005320,1.135514


In [ ]:
weekend_morning_method_2 = apriori_results_method_2(dia_da_semana = 'FDS', hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
weekend_morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
266,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.010824,0.009992,0.005828,0.538462,53.891026,0.005720,2.145018
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.011657,0.009992,0.005828,0.500000,50.041667,0.005712,1.980017
142,(CIABATTA 123G),(OVOS COM BACON E BRIE),0.019983,0.024147,0.006661,0.333333,13.804598,0.006179,1.463780
220,(CROISSANT OVOS),(SUCO DE LARANJA 400ML),0.017485,0.044963,0.007494,0.428571,9.531746,0.006708,1.671316
284,"(AGUA MINERAL S/GAS 500ML, PASTEL DE NATA 60G)",(SANDUICHE DE PEPERONI BRIE),0.009159,0.067444,0.005828,0.636364,9.435466,0.005211,2.564530
64,(CROISSANT IFOOD),(BOMBOLONI NUTELLA 80G IFOOD),0.098251,0.006661,0.005828,0.059322,8.905720,0.005174,1.055982
128,(CROISSANT OVOS),(CAFE EQUILIBRADO CAPSULA G),0.017485,0.043297,0.005828,0.333333,7.698718,0.005071,1.435054
40,(AGUA MINERAL SEM GAS 500 ML),(CROISSANT OVOS),0.044963,0.017485,0.005828,0.129630,7.413580,0.005042,1.128847
214,(SOURDOUGH 369G IFOOD),(CROISSANT IFOOD),0.015820,0.098251,0.010824,0.684211,6.963872,0.009270,2.855537
238,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.062448,0.024979,0.009992,0.160000,6.405333,0.008432,1.160739


In [ ]:
weekday_afternoon_method_2 = apriori_results_method_2(dia_da_semana = 'DDS', hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [ ]:
weekday_afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
146,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.006613,0.005878,0.005143,0.777778,132.319444,0.005104,4.473549
147,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.005143,0.008082,0.005143,1.000000,123.727273,0.005102,inf
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.008082,0.005878,0.005878,0.727273,123.727273,0.005831,3.645114
114,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.033064,0.014695,0.006613,0.200000,13.610000,0.006127,1.231631
153,"(PASTEL DE NATA 60G, BOMBOLONI NUTELLA 80G)",(FOLHADO FRANGO 160G),0.008082,0.053637,0.005143,0.636364,11.864259,0.004710,2.602498
154,"(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)",(BOMBOLONI NUTELLA 80G),0.013226,0.046289,0.005143,0.388889,8.401235,0.004531,1.560617
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.161646,0.005878,0.005143,0.031818,5.413068,0.004193,1.026793
42,(CAFE INTENSO CAPSULA P 50ML),(CAFE EQUILIBRADO CAPSULA P),0.032329,0.043350,0.007348,0.227273,5.242681,0.005946,1.238017
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.161646,0.008082,0.006613,0.040909,5.061570,0.005306,1.034227
16,(BOMBOLONI DOCE DE LEITE 80G),(BOMBOLONI NUTELLA 80G),0.050698,0.046289,0.011756,0.231884,5.009432,0.009409,1.241623


In [ ]:
weekday_afternoon_method_2.groupby(by='antecedents')['lift'].sum().sort_values(ascending=False)

antecedents
(CROISSANT, ADICIONAL DE MUSSARELA)             132.319444
(CROISSANT, ADICIONAL DE PRESUNTO)              123.727273
(ADICIONAL DE MUSSARELA)                        123.727273
(CROISSANT)                                      37.912645
(CROISSANT IFOOD)                                16.151550
(BOMBOLONI DOCE DE LEITE 80G)                    12.691972
(PASTEL DE NATA 60G, BOMBOLONI NUTELLA 80G)      11.864259
(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)        10.807043
(CAFE INTENSO CAPSULA P 50ML)                    10.219536
(COOKIES 80G)                                     9.847122
(COCA COLA LATA 350 ML)                           9.323056
(MINI CROISSANT 25G KIT C 5)                      8.989524
(FOLHADO MACA 80G)                                7.960610
(CROISSANT  TRADICIONAL 80 G)                     5.450213
(PASTEL DE NATA 60G)                              5.177662
(FOLHADO FRANGO 160G)                             4.534874
(FOLHADO FRANGO 160G, BOMBOLONI NUTELLA 80G)